## NYC RESTAURANT INSPECTION RESULTS 

* Student Name: Andrea Cabello
* Full Time Online Student
* Instructor Name:Rafael Carrasco

### I. Overview

* The dataset contains every sustained or not yet adjudicated violation citation from every inspection conducted up to three years prior to the most recent inspection for restaurants and college cafeterias in an active status on the RECORD DATE (11/04/2020). 
* When an inspection results in more than one violation, values for associated fields are repeated for each additional violation record. 
* Establishments are uniquely identified by their CAMIS (record ID) number. Keep in mind that thousands of restaurants start business and go out of business every year; only restaurants in an active status are included in the dataset.
* Records are also included for each restaurant that has applied for a permit but has not yet been inspected and for inspections resulting in no violations. Establishments with inspection date of 1/1/1900 are new establishments that have not yet received an inspection. 
* Restaurants that received no violations are represented by a single row and coded as having no violations using the ACTION field.
* Because this dataset is compiled from several large administrative data systems, it contains some illogical values that could be a result of data entry or transfer errors. Data may also be missing.
* This dataset and the information on the Health Department’s Restaurant Grading website come from the same data source. The Health Department’s Restaurant Grading website is here:
http://www1.nyc.gov/site/doh/services/restaurant-grades.page
* The Health Department's Violation Description is here: https://www1.nyc.gov/assets/doh/downloads/pdf/rii/enforcement-guidelines.pdf
* See the data dictionary file in the Data folder for a summary of data fields and allowable values.

### II. Business Problem

Let's start with our usual preliminary list of questions to get a better idea of what type of insight we could obtain from our data.
* What violations are committed the most?  
* What Businesses are falling in a violation the most? Number of Violations by DBA
* Number of Violation by Camis.
* Inspection type: How many inspections does a business get? 
* Count of type of inspection visualization x= type of inspection y=q
* Critical Flag
* Which Boro has the most violations in record? How about Violations by Council?
* Relationship between total number of violations and critical flag
* Total Violations per year
* How many restaurants had X number of violations
* Build a chart x = grade, y = total of restaurants per grade.
* Action taken by the DOH visualization.


In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('../Data/DOHMH_New_York_City_Restaurant_Inspection_Results.zip')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400146 entries, 0 to 400145
Data columns (total 26 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CAMIS                  400146 non-null  int64  
 1   DBA                    399621 non-null  object 
 2   BORO                   400146 non-null  object 
 3   BUILDING               398908 non-null  object 
 4   STREET                 400061 non-null  object 
 5   ZIPCODE                393418 non-null  float64
 6   PHONE                  400126 non-null  object 
 7   CUISINE DESCRIPTION    400146 non-null  object 
 8   INSPECTION DATE        400146 non-null  object 
 9   ACTION                 397402 non-null  object 
 10  VIOLATION CODE         393116 non-null  object 
 11  VIOLATION DESCRIPTION  389746 non-null  object 
 12  CRITICAL FLAG          389746 non-null  object 
 13  SCORE                  382023 non-null  float64
 14  GRADE                  202256 non-nu

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,40750285,CROSSROADS (MARRIOTT MARQUIS),Manhattan,1535,BROADWAY,10036.0,2123981900,American,11/02/2018,Violations were cited in the following area(s).,...,11/04/2020,Cycle Inspection / Initial Inspection,40.758260,-73.985468,105.0,3.0,11900.0,1024727.0,1.010170e+09,MN17
1,50053206,PESCADA NEW YORK,Brooklyn,1776,OCEAN AVENUE,11230.0,7187662759,Jewish/Kosher,08/16/2018,Violations were cited in the following area(s).,...,11/04/2020,Pre-permit (Operational) / Compliance Inspection,40.618980,-73.955129,314.0,48.0,53800.0,3180746.0,3.067390e+09,BK43
2,50055882,GARDENIA TERRACE,Manhattan,826,10 AVENUE,10019.0,2123340489,Mediterranean,10/09/2018,Violations were cited in the following area(s).,...,11/04/2020,Cycle Inspection / Re-inspection,40.767615,-73.989635,104.0,3.0,13900.0,1026807.0,1.010640e+09,MN15
3,50054452,CHICK-FIL-A QUEENS CENTER MALL,Queens,90-15,QUEENS BOULEVARD,11373.0,7182711755,American,01/09/2018,Violations were cited in the following area(s).,...,11/04/2020,Cycle Inspection / Initial Inspection,40.733840,-73.871577,404.0,25.0,68300.0,4045999.0,4.018600e+09,QN29
4,50079461,TEPPEN RAMEN,Manhattan,159,EAST 55 STREET,10022.0,2123552974,Japanese,12/20/2018,Violations were cited in the following area(s).,...,11/04/2020,Pre-permit (Operational) / Second Compliance I...,40.759346,-73.969415,106.0,4.0,10000.0,1036532.0,1.013100e+09,MN19


Let's begin by dropping the columns we don't think will be of use.

In [3]:
df = df.drop(columns=[
    'PHONE', 'BUILDING', 'STREET', 'RECORD DATE', 'GRADE DATE',
    'Community Board', 'Census Tract', 'BIN', 'BBL', 'NTA',
    'SCORE'])

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400146 entries, 0 to 400145
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CAMIS                  400146 non-null  int64  
 1   DBA                    399621 non-null  object 
 2   BORO                   400146 non-null  object 
 3   ZIPCODE                393418 non-null  float64
 4   CUISINE DESCRIPTION    400146 non-null  object 
 5   INSPECTION DATE        400146 non-null  object 
 6   ACTION                 397402 non-null  object 
 7   VIOLATION CODE         393116 non-null  object 
 8   VIOLATION DESCRIPTION  389746 non-null  object 
 9   CRITICAL FLAG          389746 non-null  object 
 10  GRADE                  202256 non-null  object 
 11  INSPECTION TYPE        397402 non-null  object 
 12  Latitude               398534 non-null  float64
 13  Longitude              398534 non-null  float64
 14  Council District       391927 non-nu

Now, let's get rid of all null values and work on the format of the column names.

In [5]:
df = df.dropna()
df = df.rename(columns={'CUISINE DESCRIPTION': 'Cuisine'})
df.columns = df.columns.str.title()
df.columns = df.columns.str.replace(' ', '')
df['Camis'] = df.Camis.astype('str')
df['Zipcode']= df.Zipcode.astype('str')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 196162 entries, 2 to 400145
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Camis                 196162 non-null  object 
 1   Dba                   196162 non-null  object 
 2   Boro                  196162 non-null  object 
 3   Zipcode               196162 non-null  object 
 4   Cuisine               196162 non-null  object 
 5   InspectionDate        196162 non-null  object 
 6   Action                196162 non-null  object 
 7   ViolationCode         196162 non-null  object 
 8   ViolationDescription  196162 non-null  object 
 9   CriticalFlag          196162 non-null  object 
 10  Grade                 196162 non-null  object 
 11  InspectionType        196162 non-null  object 
 12  Latitude              196162 non-null  float64
 13  Longitude             196162 non-null  float64
 14  CouncilDistrict       196162 non-null  float64
dtype

Now that we have our data mostly clean, let's take a closer look at our Features and Target.

### FEATURES

### * Camis 
Unique ID number per restaurant permit.

In [7]:
df['Camis'].value_counts().describe()

count    25247.000000
mean         7.769715
std          4.315377
min          1.000000
25%          5.000000
50%          7.000000
75%         10.000000
max         46.000000
Name: Camis, dtype: float64

### * Dba 
Commercial name used by the establishment(doing business as).

In [8]:
df['Dba'].value_counts().describe()

count    20316.000000
mean         9.655542
std         30.535983
min          1.000000
25%          5.000000
50%          8.000000
75%         11.000000
max       2789.000000
Name: Dba, dtype: float64

In [9]:
df['Dba'].nunique()

20316

In [10]:
df['Dba'] = df['Dba'].replace(
    to_replace=["DUNKIN', BASKIN ROBBINS", "DUNKIN' DONUTS", 'DUNKIN'], value="DUNKIN'")

df['Dba'] = df['Dba'].replace(
    to_replace=["STARBUCKS COFFEE"], value="STARBUCKS")

df['Dba'] = df['Dba'].replace(
    to_replace=["MCDONALD'S"], value="MCDONALDS")

df['Dba'] = df['Dba'].replace(
    to_replace=["GOLDEN KRUST CARIBBEAN BAKERY & GRILL"], value="GOLDEN KRUST")

df['Dba'] = df['Dba'].replace(
    to_replace=["DOS TOROS"], value="DOS TOROS TAQUERIA")

* The raw data presented contains one row per violation and there are 196162 total violations by 25247 unique restaurant establishments (Camis). 
* What if I summarize the total violations committed by each restaurant (Camis) and by Dba in  new features. 
* So I will have 'totalViolations_Dba'(continuous), 'TotalViolations_Camis'(continuous), 'Boro' (5categories),  'action'(5possible actions taken by the doh), 'cuisin type', 'grade'. 

### * Grade
Every inspection results in a Grade. 

In [11]:
df['Grade'].value_counts()
#A, B, C, P, GROUP[Z,N,G] = 3795

A    154853
B     25547
C      9661
N      2480
P      2390
Z      1225
G         6
Name: Grade, dtype: int64

After reviewing the DataSet Description, I decided to group 'Z', 'G' and 'N' as "Not Yet Graded" and leave 'P' as "Grade Pending issued on re-opening following an initial inspection that resulted in a closure".

In [12]:
df['Grade'] = df['Grade'].replace(to_replace=['Z', 'G'], value='N')

As a result we have 5 classes and an obvious class imbalance, which in this case is natural and expected since we want every restaurant to get GRADE 'A'.

In [13]:
df['Grade'].value_counts()

A    154853
B     25547
C      9661
N      3711
P      2390
Name: Grade, dtype: int64

### * Boro 
There are 5 boros in New York City:
Manhattan · 2. Brooklyn · 3. Queens · 4. The Bronx · 5. Staten Island.

In [14]:
df.Boro.value_counts()

Manhattan        76532
Brooklyn         50007
Queens           44790
Bronx            18051
Staten Island     6782
Name: Boro, dtype: int64

1. Manhattan · 2. Brooklyn · 3. Queens · 4. The Bronx · 5. Staten Island.

### * Cuisine

In [15]:
df.Cuisine.value_counts()

American              42148
Chinese               20119
Café/Coffee/Tea       10998
Pizza                  8676
Mexican                7950
                      ...  
Nuts/Confectionary       27
Czech                    26
Southwestern             20
Chilean                  17
Basque                    6
Name: Cuisine, Length: 84, dtype: int64

In [16]:
df.Cuisine.unique()

array(['Mediterranean', 'American', 'Pizza/Italian', 'Italian',
       'Australian', 'Pizza', 'Chinese/Japanese', 'Chinese', 'Bakery',
       'Café/Coffee/Tea', 'Seafood',
       'Latin (Cuban, Dominican, Puerto Rican, South & Central American)',
       'Donuts', 'Tapas', 'Thai', 'Mexican', 'Korean', 'Chicken',
       'Indian', 'Hamburgers', 'Asian', 'Spanish', 'Irish', 'Peruvian',
       'Jewish/Kosher', 'Caribbean', 'Ice Cream, Gelato, Yogurt, Ices',
       'Greek', 'Armenian', 'Delicatessen',
       'Juice, Smoothies, Fruit Salads', 'Japanese', 'Pancakes/Waffles',
       'African', 'Brazilian', 'Tex-Mex', 'French', 'Eastern European',
       'Middle Eastern', 'Sandwiches', 'Soups & Sandwiches',
       'Bagels/Pretzels',
       'Bottled beverages, including water, sodas, juices, etc.',
       'Soul Food', 'Vietnamese/Cambodian/Malaysia', 'Pakistani',
       'Polish', 'Bangladeshi', 'Other', 'Moroccan',
       'Sandwiches/Salads/Mixed Buffet', 'Steak', 'Hotdogs', 'Turkish',
       'Co

Below we will summarize our Cuisine Categories as much as possible. We'll try to narrow 84 into 10-15 classes.

In [17]:
df['Cuisine'] = df['Cuisine'].replace(
    to_replace=['Middle Eastern', 'Moroccan', 'Tapas', 'Spanish', 'Greek',
                'Armenian', 'Basque', 'Ethiopian', 'Afghan', 'Iranian', 
                'Egyptian', 'Turkish', 'Pakistani', 'Eastern European'], 
                value='Mediterranean')

df['Cuisine'] = df['Cuisine'].replace(
    to_replace=['Australian', 'Scandinavian', 'Czech', 'English', 'German', 
                'Portuguese', 'Russian', 'Polish', 'Continental', 'Irish',
                'French', 'Hawaiian'], value='International')

df['Cuisine'] = df['Cuisine'].replace(
    to_replace=['Pizza/Italian'], value='Pizza')

df['Cuisine'] = df['Cuisine'].replace(
    to_replace=['Chinese', 'Japanese','Chinese/Japanese', 'Filipino', 'Indian', 
                'Indonesian', 'Bangladeshi','Thai', 'Korean', 'Chinese/Cuban', 
                'Vietnamese/Cambodian/Malaysia'], value='Asian')

df['Cuisine'] = df['Cuisine'].replace(
    to_replace=['Bottled beverages, including water, sodas, juices, etc.', 
                'Café/Coffee/Tea'], value='Coffee/Beverages')

df['Cuisine'] = df['Cuisine'].replace(
    to_replace=['Bagels/Pretzels', 'Donuts'], value='Bakery')

df['Cuisine'] = df['Cuisine'].replace(
    to_replace=['Chilean','Peruvian', 'Caribbean', 'Brazilian',
                'Latin (Cuban, Dominican, Puerto Rican, South & Central American)'],
                value='Latin/Caribbean') 

df['Cuisine'] = df['Cuisine'].replace(
    to_replace=['Ice Cream, Gelato, Yogurt, Ices','Seafood', 'Chicken', 
                'Nuts/Confectionary', 'Hotdogs/Pretzels', 'Hotdogs',
                'Fruits/Vegetables', 'Vegetarian', 'Salads',
                'Steak', 'Juice, Smoothies, Fruit Salads',  
                'Pancakes/Waffles',  'Jewish/Kosher'], value='Specialty') 

df['Cuisine'] = df['Cuisine'].replace(
    to_replace=['Sandwiches', 'Soups', 'Soup/Sandwiches'], 
                value='Soups & Sandwiches') 

df['Cuisine'] = df['Cuisine'].replace(
    to_replace=['Delicatessen', 'Sandwiches/Salads/Mixed Buffet'],
                value='Deli/Buffett')

df['Cuisine'] = df['Cuisine'].replace(
    to_replace=['Soul Food', 'Californian', 'Creole/Cajun', 'Cajun', 
                'Creole', 'Barbecue', 'Tex-Mex', 'Southwestern', 'Hamburgers'], 
                value='American')     

In [18]:
df.Cuisine.value_counts()

American                     47034
Asian                        40401
Latin/Caribbean              15164
Specialty                    14976
Pizza                        12656
Mediterranean                12341
Coffee/Beverages             11662
Bakery                       11185
Mexican                       7950
Italian                       7849
International                 6102
Deli/Buffett                  4507
Soups & Sandwiches            2741
Other                          841
African                        699
Not Listed/Not Applicable       54
Name: Cuisine, dtype: int64

### * Action 
Action taken by the NYS Department of Health after the inspection.

In [19]:
df.Action.value_counts()

Violations were cited in the following area(s).                                                                                        192365
Establishment re-opened by DOHMH                                                                                                         3701
Establishment Closed by DOHMH.  Violations were cited in the following area(s) and those requiring immediate action were addressed.        88
Establishment re-closed by DOHMH                                                                                                            8
Name: Action, dtype: int64

In [20]:
df.Action.nunique()

4

### * Inspection Type
Most interested on Cycle Inspections: Re-inspection, Reopening Inspection and Initial Inspection.

In [21]:
df['InspectionType'].value_counts()

Cycle Inspection / Re-inspection                        89894
Cycle Inspection / Initial Inspection                   80510
Pre-permit (Operational) / Re-inspection                11449
Pre-permit (Operational) / Initial Inspection            9982
Cycle Inspection / Reopening Inspection                  3150
Pre-permit (Operational) / Reopening Inspection           559
Pre-permit (Non-operational) / Initial Inspection         522
Pre-permit (Non-operational) / Re-inspection               67
Inter-Agency Task Force / Initial Inspection               14
Pre-permit (Non-operational) / Compliance Inspection        8
Pre-permit (Operational) / Compliance Inspection            5
Administrative Miscellaneous / Initial Inspection           2
Name: InspectionType, dtype: int64

We have to deal with the unique values. To avoid dropping valuable data. We will reasign them into more global categories.

In [22]:
df['InspectionType'] = df['InspectionType'].replace(
    to_replace=['Pre-permit (Operational) / Compliance Inspection'], 
    value='Pre-permit (Operational) / Re-inspection')

df['InspectionType'] = df['InspectionType'].replace(
    to_replace=['Pre-permit (Non-operational) / Compliance Inspection', 
                'Pre-permit (Non-operational) / Re-inspection', 
               'Pre-permit (Non-operational) / Initial Inspection'], 
    value='Pre-permit (Non-operational)')

df['InspectionType'] = df['InspectionType'].replace(
    to_replace=['Inter-Agency Task Force / Initial Inspection', 
                'Administrative Miscellaneous / Initial Inspection'], 
    value='Cycle Inspection / Initial Inspection')

In [23]:
df['InspectionType'].value_counts()

Cycle Inspection / Re-inspection                   89894
Cycle Inspection / Initial Inspection              80526
Pre-permit (Operational) / Re-inspection           11454
Pre-permit (Operational) / Initial Inspection       9982
Cycle Inspection / Reopening Inspection             3150
Pre-permit (Non-operational)                         597
Pre-permit (Operational) / Reopening Inspection      559
Name: InspectionType, dtype: int64

### * Inspection Date

In [24]:
df['InspectionYear'] = df['InspectionDate'].apply(lambda x: x[6:10])

In [25]:
df['InspectionYear'].value_counts()

2019    72159
2018    60153
2017    38075
2020    17032
2016     8651
2015       65
2014       18
2012        6
2013        3
Name: InspectionYear, dtype: int64

### * ViolationCode
Unique Violation ID

In [26]:
df['ViolationCode'].unique()

array(['04A', '10F', '04C', '06D', '04L', '04N', '08A', '06C', '09C',
       '06E', '02B', '06A', '09B', '10B', '06B', '02G', '04H', '03C',
       '04O', '06F', '04J', '10E', '10I', '04M', '05D', '10D', '08C',
       '05F', '02I', '10H', '02H', '04E', '10J', '02A', '02C', '10A',
       '04F', '02D', '10G', '09A', '04K', '05A', '03B', '10C', '03A',
       '05C', '04B', '04D', '05H', '08B', '04G', '06G', '03G', '06I',
       '05B', '05E', '02F', '03D', '07A', '06H', '18C', '20D', '03E',
       '04I', '02E'], dtype=object)

In [27]:
x = df['ViolationCode'].value_counts()

In [28]:
df['ViolationCode'].value_counts().describe()

count       65.000000
mean      3017.876923
std       6397.219580
min          1.000000
25%         34.000000
50%        292.000000
75%       3047.000000
max      40490.000000
Name: ViolationCode, dtype: float64

### * Council District
With this feature I am hoping to obtain info about which Council Districts need to enforce the Health Code the most. 

In [29]:
df['CouncilDistrict'].nunique()

51

In [30]:
df['CouncilDistrict'].unique()

array([ 3., 25., 39., 22.,  4.,  2.,  1.,  6., 21., 32., 17., 20., 38.,
       24., 10., 43., 34., 50., 42., 19., 30., 29., 47., 11.,  8., 12.,
       35., 26.,  9., 33., 41.,  5., 48., 36., 46., 45., 40., 27., 16.,
       15., 51., 28., 13., 49.,  7., 44., 37., 14., 31., 18., 23.])

### * IsChain

It will be important to differentiate if a business is a fastfood chain or not. As a restauranteur, I understand  that a chain is any eatery with at least 5 locations, or at least one that crosses state borders.
Up next, I will create list containing all the chain names in our data set.

In [31]:
chain_restaurants = ['DON ALEX RESTAURANT', 'COCO FRESH TEA & JUICE', 'TOASTIES', 'DOUBLE DRAGON',
                     'YUMMY TACO', "JIMBO'S HAMBURGER PALACE","LUKE'S LOBSTER", 'EMPANADAS MONUMENTAL',
                     "FISHERMAN'S COVE", 'CAFFE BENE', 'LA ISLA RESTAURANT', 'OAXACA TAQUERIA', 'TBAAR',
                     'CHIRPING CHICKEN', 'SPICE', 'CHINA WOK', "AUNTIE ANNE'S PRETZELS", 'BLUE BOTTLE COFFEE',
                     'OUTBACK STEAKHOUSE', 'MCDONALDS', "XI'AN FAMOUS FOODS", "JOHN'S FRIED CHICKEN",
                     "KENNEDY FRIED CHICKEN & PIZZA", "ARTICHOKE BASILLE'S PIZZA", "TASTE OF CHINA","DALLAS BBQ",
                     "DOMINO'S PIZZA", 'GREGORYS COFFEE', 'FAY DA BAKERY', 'PIZZA HUT', 'GONG CHA', 
                     "DUNKIN'", "SUBWAY", 'STARBUCKS', 'KENNEDY FRIED CHICKEN', 'BURGER KING', 'POPEYES', 
                     'CROWN FRIED CHICKEN', 'CHIPOTLE MEXICAN GRILL', 'GOLDEN KRUST', 'INSOMNIA COOKIES',
                     'KFC', "WENDY'S", "DOMINO'S", 'PRET A MANGER', 'CHECKERS', 'LE PAIN QUOTIDIEN', 
                     'CARVEL ICE CREAM', 'VIVI BUBBLE TEA', 'JUST SALAD', 'BOSTON MARKET', "PAPA JOHN'S",
                     'LITTLE CAESARS', 'IHOP', "APPLEBEE'S", "CHOP'T", 'SHAKE SHACK', 'LENWICH', 'TACO BELL',
                     'PARIS BAGUETTE', 'TEXAS CHICKEN & BURGERS', 'FIVE GUYS FAMOUS BURGERS AND FRIES', 
                     'KUNG FU TEA', 'PANERA BREAD', 'WHITE CASTLE', 'AU BON PAIN', 'HALE & HEARTY SOUP',
                     'JOE & THE JUICE', 'BAREBURGER', 'BREAD & BUTTER', 'FRESH & CO', 'POTBELLY SANDWICH WORKS', 
                     "CHUCK E. CHEESE'S", 'AMC THEATRES', 'BLUESTONE LANE','HAPPY GARDEN', 'SWEETGREEN', 
                     'NO. 1 CHINESE RESTAURANT', 'MAISON KAYSER', 'DIG INN', 'JUICE GENERATION', 'RED LOBSTER',
                     "GREGORY'S COFFEE", "JOE'S PIZZA", 'PRONTO PIZZA', 'CARIDAD RESTAURANT', "PAPA JOHN'S PIZZA",
                     'LITTLE ITALY PIZZA', 'THINK COFFEE', 'COLD STONE CREAMERY', 'NATIONAL BAKERY', 'RED MANGO',
                     'BARNES & NOBLE CAFE', 'LA BAGEL DELIGHT', "SOPHIE'S CUBAN CUISINE", "WINGSTOP", 
                     'TOUS LES JOURS', 'MAMAN', 'BELLA NAPOLI', 'DOS TOROS TAQUERIA', 'PELICANA CHICKEN',
                     'MAGNOLIA BAKERY', 'WESTVILLE']

Using the .isin() method, I will create a new feature that will tell us if all other values in each row are for a Chain or Non-Chain establisment.

In [32]:
df['IsChain'] = df['Dba'].isin(chain_restaurants)

In [33]:
df.IsChain.value_counts()

False    174382
True      21780
Name: IsChain, dtype: int64

### TARGET
'CriticalFlag' is an indicator of a critical violation that is most likely to contribute to foodborne illness. Our possible values are Yes and No (binary).

In [34]:
df['CriticalFlag'].value_counts()

Y    99321
N    96841
Name: CriticalFlag, dtype: int64

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 196162 entries, 2 to 400145
Data columns (total 17 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Camis                 196162 non-null  object 
 1   Dba                   196162 non-null  object 
 2   Boro                  196162 non-null  object 
 3   Zipcode               196162 non-null  object 
 4   Cuisine               196162 non-null  object 
 5   InspectionDate        196162 non-null  object 
 6   Action                196162 non-null  object 
 7   ViolationCode         196162 non-null  object 
 8   ViolationDescription  196162 non-null  object 
 9   CriticalFlag          196162 non-null  object 
 10  Grade                 196162 non-null  object 
 11  InspectionType        196162 non-null  object 
 12  Latitude              196162 non-null  float64
 13  Longitude             196162 non-null  float64
 14  CouncilDistrict       196162 non-null  float64
 15  

In [36]:
X = df.drop(columns=['CriticalFlag'])
y = df['CriticalFlag']

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 42)

In [38]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117697 entries, 126620 to 248657
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Camis                 117697 non-null  object 
 1   Dba                   117697 non-null  object 
 2   Boro                  117697 non-null  object 
 3   Zipcode               117697 non-null  object 
 4   Cuisine               117697 non-null  object 
 5   InspectionDate        117697 non-null  object 
 6   Action                117697 non-null  object 
 7   ViolationCode         117697 non-null  object 
 8   ViolationDescription  117697 non-null  object 
 9   Grade                 117697 non-null  object 
 10  InspectionType        117697 non-null  object 
 11  Latitude              117697 non-null  float64
 12  Longitude             117697 non-null  float64
 13  CouncilDistrict       117697 non-null  float64
 14  InspectionYear        117697 non-null  object 


In [39]:
%store X_train 

Stored 'X_train' (DataFrame)


In [40]:
%store y_train

Stored 'y_train' (Series)


In [41]:
%store X_test

Stored 'X_test' (DataFrame)


In [42]:
%store y_test

Stored 'y_test' (Series)
